In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV


import statsmodels.api as sm  

from sklearn.linear_model import Lasso

from sklearn.metrics import mean_absolute_error,r2_score

In [2]:
train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [3]:
attributes = train.drop('SalePrice', axis=1)
labels = train.SalePrice

In [4]:
more_than_zero_null_values = attributes.isnull().sum()>0

attributes.isnull().sum()[more_than_zero_null_values]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [5]:
num_features = attributes.select_dtypes(include = 'number').columns.tolist()
print(f"The numerical features are {len(num_features)}.")
print(f"They are {', '.join(num_features)}.")

The numerical features are 37.
They are Id, MSSubClass, LotFrontage, LotArea, OverallQual, OverallCond, YearBuilt, YearRemodAdd, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageYrBlt, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal, MoSold, YrSold.


In [6]:
cat_features =  attributes.select_dtypes(exclude = 'number').columns.tolist()
print(f"The categorical features are {len(cat_features)}.")
print(f"They are {', '.join(cat_features)}.")

The categorical features are 43.
They are MSZoning, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition.


In [7]:
numeric_pipeline = Pipeline(
    steps = [
        ('Numerical imputer', SimpleImputer(strategy='mean')),
        ('numerical scaler', MinMaxScaler()) 
    ])

categorical_pipeline = Pipeline(
    steps = [
        ('categorical imputer', SimpleImputer(strategy='most_frequent')),
        ('categorical one-hot', OneHotEncoder(handle_unknown='ignore',sparse = False)) 
    ])

In [8]:
full_transformer = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, num_features),
    ('category', categorical_pipeline, cat_features)
])

In [9]:
a = full_transformer.fit_transform(attributes)
a

array([[0.00000000e+00, 2.35294118e-01, 1.50684932e-01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.85400960e-04, 0.00000000e+00, 2.02054795e-01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.37080192e-03, 2.35294118e-01, 1.60958904e-01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [9.98629198e-01, 2.94117647e-01, 1.54109589e-01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [9.99314599e-01, 0.00000000e+00, 1.60958904e-01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 1.84931507e-01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [10]:
a = np.append(arr = np.ones((1460,1)).astype(int),values = a,axis = 1)


In [14]:
regressor_OLS=sm.OLS(endog = labels, exog=a).fit()  

In [15]:
regressor_OLS.summary()  

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     68.77
Date:                Sat, 03 Dec 2022   Prob (F-statistic):               0.00
Time:                        12:27:23   Log-Likelihood:                -16580.
No. Observations:                1460   AIC:                         3.365e+04
Df Residuals:                    1216   BIC:                         3.494e+04
Df Model:                         243                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.671e+04   3420.255     -4.884      0.000   -2.34e+04   -9995.319
x1          1353.5218   2251.953      0.601      0.548   -3064.622    5771.666
x2          -1.02e+04   1.41e+04     -0.725      0.469   -3.78e+04    1.74e+04
x3          1.165e+04   1.28e+04      0.909      0.364   -1.35e+04    3.68e+04
x4          1.561e+05   2.33e+04      6.687      0.000     1.1e+05    2.02e+05
x5          5.827e+04   9110.819      6.396      0.000    4.04e+04    7.61e+04
x6          4.632e+04   6972.130      6.644      0.000    3.26e+04       6e+04
x7          4.283e+04   1.05e+04      4.074      0.000    2.22e+04    6.35e+04
x8          5826.4378   3336.864      1.746      0.081    -720.211    1.24e+04
x9           3.42e+04   9235.692      3.703      0.000    1.61e+04    5.23e+04
x10         1.006e+05   1.37e+04      7.323      0.000    7.37e+04    1.28e+05
x11         1.338e+04   1.13e+04      1.187      0.235   -8728.963    3.55e+04
x12        -2498.2391   5916.695     -0.422      0.673   -1.41e+04    9109.824
x13         9.521e+04   1.38e+04      6.887      0.000    6.81e+04    1.22e+05
x14          1.14e+05   1.41e+04      8.080      0.000    8.63e+04    1.42e+05
x15         8.286e+04   1.05e+04      7.879      0.000    6.22e+04    1.03e+05
x16        -1.325e+04   1.06e+04     -1.255      0.210    -3.4e+04    7470.415
x17         1.244e+05   1.12e+04     11.157      0.000    1.03e+05    1.46e+05
x18         3251.6239   5922.440      0.549      0.583   -8367.710    1.49e+04
x19        -1746.1627   6054.501     -0.288      0.773   -1.36e+04    1.01e+04
x20          1.21e+04   6605.661      1.832      0.067    -859.534    2.51e+04
x21         3493.3844   4189.541      0.834      0.405   -4726.146    1.17e+04
x22        -2.877e+04   1.09e+04     -2.635      0.009   -5.02e+04   -7349.118
x23        -3.964e+04    1.7e+04     -2.335      0.020   -7.29e+04   -6338.246
x24         1.931e+04   1.15e+04      1.684      0.092   -3190.551    4.18e+04
x25         8240.0408   4573.973      1.802      0.072    -733.713    1.72e+04
x26         2578.7072   6242.766      0.413      0.680   -9669.080    1.48e+04
x27         1.274e+04   8781.530      1.450      0.147   -4493.035       3e+04
x28          1.93e+04    1.1e+04      1.756      0.079   -2264.627    4.09e+04
x29         1.327e+04   5017.128      2.645      0.008    3425.699    2.31e+04
x30         3292.7583   6272.026      0.525      0.600   -9012.435    1.56e+04
x31         1279.4230   6872.175      0.186      0.852   -1.22e+04    1.48e+04
x32         1.675e+04   1.14e+04      1.473      0.141   -5554.487    3.91e+04
x33          1.65e+04   6005.009      2.748      0.006    4723.058    2.83e+04
x34         7.118e+04   2.16e+04      3.289      0.001    2.87e+04    1.14e+05
x35         5.325e+04   6.21e+04      0.857      0.392   -6.87e+04    1.75e+05
x3

In [ ]:
a = [:,[]]

In [ ]:
regressor_OLS=sm.OLS(endog = labels, exog=a).fit()  